In [1]:
import pandas as pd
import ast
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.corpus import stopwords
import string
from gensim.models import Word2Vec,KeyedVectors
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('../data/tmdb_5000_movies.csv', index_col=False)

In [3]:
df.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count'],
      dtype='object')

In [4]:
pd.set_option('display.max_columns', df.shape[1])

In [5]:
df = df[['genres','keywords','original_title','overview','tagline']]

In [6]:
df.head()

,genres,keywords,original_title,overview,tagline
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",Avatar,"In the 22nd century, a paraplegic Marine is di...",Enter the World of Pandora.
1,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","At the end of the world, the adventure begins."
2,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",Spectre,A cryptic message from Bond’s past sends him o...,A Plan No One Escapes
3,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",The Dark Knight Rises,Following the death of District Attorney Harve...,The Legend Ends
4,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",John Carter,"John Carter is a war-weary, former military ca...","Lost in our world, found in another."


In [7]:
df.genres = df.genres.apply(lambda x:ast.literal_eval(x))
df.keywords = df.keywords.apply(lambda x:ast.literal_eval(x))

In [8]:
df.loc[0,['genres','keywords']]

genres      [{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...
keywords    [{'id': 1463, 'name': 'culture clash'}, {'id':...
Name: 0, dtype: object

In [9]:
df.genres = df.genres.apply(lambda x:', '.join(list(map(lambda y:y['name'],x))))
df.keywords = df.keywords.apply(lambda x:', '.join(list(map(lambda y:y['name'],x))))

In [10]:
df.fillna('',inplace=True)

In [11]:
df['doc'] = df.genres+' '+df.keywords+' '+df.overview+' '+df.tagline # you can experiment with which features to take

In [12]:
df

,genres,keywords,original_title,overview,tagline,doc
0,"Action, Adventure, Fantasy, Science Fiction","culture clash, future, space war, space colony...",Avatar,"In the 22nd century, a paraplegic Marine is di...",Enter the World of Pandora.,"Action, Adventure, Fantasy, Science Fiction cu..."
1,"Adventure, Fantasy, Action","ocean, drug abuse, exotic island, east india t...",Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","At the end of the world, the adventure begins.","Adventure, Fantasy, Action ocean, drug abuse, ..."
2,"Action, Adventure, Crime","spy, based on novel, secret agent, sequel, mi6...",Spectre,A cryptic message from Bond’s past sends him o...,A Plan No One Escapes,"Action, Adventure, Crime spy, based on novel, ..."
3,"Action, Crime, Drama, Thriller","dc comics, crime fighter, terrorist, secret id...",The Dark Knight Rises,Following the death of District Attorney Harve...,The Legend Ends,"Action, Crime, Drama, Thriller dc comics, crim..."
4,"Action, Adventure, Science Fiction","based on novel, mars, medallion, space travel,...",John Carter,"John Carter is a war-weary, former military ca...","Lost in our world, found in another.","Action, Adventure, Science Fiction based on no..."
...,...,...,...,...,...,...
4798,"Action, Crime, Thriller","united states–mexico barrier, legs, arms, pape...",El Mariachi,El Mariachi just wants to play his guitar and ...,"He didn't come looking for trouble, but troubl...","Action, Crime, Thriller united states–mexico b..."
4799,"Comedy, Romance",,Newlyweds,A newlywed couple's honeymoon is upended by th...,A newlywed couple's honeymoon is upended by th...,"Comedy, Romance A newlywed couple's honeymoon..."
4800,"Comedy, Drama, Romance, TV Movie","date, love at first sight, narration, investig...","Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...",,"Comedy, Drama, Romance, TV Movie date, love at..."
4801,,,Shanghai Calling,When ambitious New York attorney Sam is sent t...,A New Yorker in Shanghai,When ambitious New York attorney Sam is sent...


In [13]:
da = df[['doc','original_title']]

In [14]:
da

,doc,original_title
0,"Action, Adventure, Fantasy, Science Fiction cu...",Avatar
1,"Adventure, Fantasy, Action ocean, drug abuse, ...",Pirates of the Caribbean: At World's End
2,"Action, Adventure, Crime spy, based on novel, ...",Spectre
3,"Action, Crime, Drama, Thriller dc comics, crim...",The Dark Knight Rises
4,"Action, Adventure, Science Fiction based on no...",John Carter
...,...,...
4798,"Action, Crime, Thriller united states–mexico b...",El Mariachi
4799,"Comedy, Romance A newlywed couple's honeymoon...",Newlyweds
4800,"Comedy, Drama, Romance, TV Movie date, love at...","Signed, Sealed, Delivered"
4801,When ambitious New York attorney Sam is sent...,Shanghai Calling


In [15]:
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else :
        return wordnet.NOUN

In [16]:
da.head()

,doc,original_title
0,"Action, Adventure, Fantasy, Science Fiction cu...",Avatar
1,"Adventure, Fantasy, Action ocean, drug abuse, ...",Pirates of the Caribbean: At World's End
2,"Action, Adventure, Crime spy, based on novel, ...",Spectre
3,"Action, Crime, Drama, Thriller dc comics, crim...",The Dark Knight Rises
4,"Action, Adventure, Science Fiction based on no...",John Carter


In [17]:
da.doc = da.doc.apply(lambda x:x.lower())

In [18]:
### tokenize the sentence and remove punctuations

def tokenize(sent):
    sent = nltk.word_tokenize(sent)
    sent_mod = []
    for i in sent:
        if i.isalnum():
            sent_mod.append(i)
    return sent_mod

da.doc = da.doc.apply(tokenize)

In [19]:
### Removing Stopwords,Punctuation 

stopword = stopwords.words('english')

def remove_stopwords(sent):
    sent = [i for i in sent if  i not in string.punctuation and i not in stopword]
    return sent

da.doc = da.doc.apply(remove_stopwords)

In [20]:
da.head()

,doc,original_title
0,"[action, adventure, fantasy, science, fiction,...",Avatar
1,"[adventure, fantasy, action, ocean, drug, abus...",Pirates of the Caribbean: At World's End
2,"[action, adventure, crime, spy, based, novel, ...",Spectre
3,"[action, crime, drama, thriller, dc, comics, c...",The Dark Knight Rises
4,"[action, adventure, science, fiction, based, n...",John Carter


In [21]:
### Applying stemming - using Lemmatizer

wnl = WordNetLemmatizer()

def lemmatiser(sent):
    words_and_tags = nltk.pos_tag(sent)
    final_sent = []
    for word,tags in words_and_tags:
        tag = get_wordnet_pos(tags)
        word_lemma  = wnl.lemmatize(word,pos=tag)
        final_sent.append(word_lemma)
    return final_sent

da.doc = da.doc.apply(lemmatiser)

In [22]:
da

,doc,original_title
0,"[action, adventure, fantasy, science, fiction,...",Avatar
1,"[adventure, fantasy, action, ocean, drug, abus...",Pirates of the Caribbean: At World's End
2,"[action, adventure, crime, spy, base, novel, s...",Spectre
3,"[action, crime, drama, thriller, dc, comic, cr...",The Dark Knight Rises
4,"[action, adventure, science, fiction, base, no...",John Carter
...,...,...
4798,"[action, crime, thriller, unite, barrier, legs...",El Mariachi
4799,"[comedy, romance, newlywed, couple, honeymoon,...",Newlyweds
4800,"[comedy, drama, romance, tv, movie, date, love...","Signed, Sealed, Delivered"
4801,"[ambitious, new, york, attorney, sam, send, sh...",Shanghai Calling


In [23]:
word_vectors = KeyedVectors.load_word2vec_format(
  '../models/GoogleNews-vectors-negative300.bin',
  binary=True
)

In [24]:
def get_x_vectorized(text_token):
    try:
        return word_vectors.get_mean_vector(text_token)
    except:
        return [0 for i in range(300)]

da['doc_vec'] = [get_x_vectorized(i) for i in da.doc]

In [25]:
da.head()

,doc,original_title,doc_vec
0,"[action, adventure, fantasy, science, fiction,...",Avatar,"[0.019230569, 0.023137746, 0.0097706895, 0.018..."
1,"[adventure, fantasy, action, ocean, drug, abus...",Pirates of the Caribbean: At World's End,"[0.010921266, 0.007950192, 0.0023264526, 0.035..."
2,"[action, adventure, crime, spy, base, novel, s...",Spectre,"[0.027357232, 0.009710357, 0.006613444, 0.0090..."
3,"[action, crime, drama, thriller, dc, comic, cr...",The Dark Knight Rises,"[0.014835731, 0.021145223, 0.0018086568, 0.009..."
4,"[action, adventure, science, fiction, base, no...",John Carter,"[0.022484757, 0.023881631, -0.0031604099, 0.03..."


the above applied sklearn linear kernel is nothing but dot product

In [26]:
# to get cosine similarity
from sklearn.metrics.pairwise import linear_kernel
def give_top_10_similar_movies(dz,text):
    index = dz[dz.original_title==text].index[0]
    cosine_similarities = linear_kernel(dz.doc_vec.loc[index:index+1].to_list()[0].reshape(1,-1), np.array(dz.doc_vec.to_list())).flatten()
    dz['similar'] = cosine_similarities
    return dz.sort_values('similar',ascending=False).head(10)#['original_title'].head(10).to_list()

In [27]:
give_top_10_similar_movies(da,"Newlyweds")

,doc,original_title,doc_vec,similar
4799,"[comedy, romance, newlywed, couple, honeymoon,...",Newlyweds,"[-0.00027202116, -0.0032496252, -0.060953554, ...",0.263043
2892,"[drama, romance, aftercreditsstinger, story, w...",Darling Companion,"[0.042676773, -0.040408067, -0.041340098, 0.01...",0.165855
4744,"[comedy, drama, family, mumblecore, christmas,...",Happy Christmas,"[0.011061882, -0.007447149, -0.027381068, 0.02...",0.164192
1751,"[comedy, drama, romance, secret, society, conc...",Divine Secrets of the Ya-Ya Sisterhood,"[-0.0010569064, -0.014823714, -0.018197278, 0....",0.158783
1576,"[comedy, bride, friendship, engagement, rivalr...",Bride Wars,"[-0.005093902, -0.016278673, -0.0025709332, 0....",0.156883
3610,"[drama, romance, car, accident, shift, affecti...",Not Easily Broken,"[0.034010433, -0.018088145, -0.024832822, 0.01...",0.155969
2857,"[drama, sister, independent, film, maid, honor...",Rachel Getting Married,"[-0.010069051, 0.02677975, -0.03193154, 0.0169...",0.150052
4632,"[comedy, drama, romance, woman, director, inse...",Lovely & Amazing,"[0.029310344, -0.017425818, -0.011843392, 0.02...",0.148044
1919,"[comedy, romance, successful, woman, love, try...",Un Plan parfait,"[0.0150987655, -0.015656427, -0.038264908, 0.0...",0.147105
2689,"[comedy, interracial, marriage, wedding, durin...",Our Family Wedding,"[0.0076103588, 0.011622124, -0.023433536, 0.03...",0.146601


In [28]:
give_top_10_similar_movies(da,'Scream 3')

,doc,original_title,doc_vec,similar
2282,"[crime, horror, mystery, halloween, gore, seri...",Scream,"[0.023107972, -0.020717721, 0.016121712, 0.042...",0.129586
3988,"[horror, action, science, fiction, zombie, fin...",Diary of the Dead,"[0.02204984, 0.0067375326, 0.010965145, 0.0617...",0.127921
1164,"[horror, mystery, mask, metal, detector, film,...",Scream 3,"[0.019784465, -0.0072214715, 0.019547163, 0.03...",0.126171
4186,"[thriller, comedy, horror, snuff, two, brother...",A Beginner's Guide to Snuff,"[0.0294632, 0.0045772637, 0.0041848007, 0.0356...",0.122394
2208,"[drama, horror, mystery, vampire, child, vampi...",Let Me In,"[0.05002707, 0.024396764, 0.0138470875, 0.0311...",0.121685
1824,"[thriller, drama, murder, investigation, journ...",The Face of an Angel,"[0.021877969, -0.0045519606, 0.013023631, 0.00...",0.121505
3574,"[horror, hallucination, supernatural, mirror, ...",Oculus,"[0.025278693, 0.008558375, 0.015271607, 0.0112...",0.121446
3359,"[drama, action, thriller, crime, fearless, cop...",In Too Deep,"[0.009628982, 0.016320797, 0.012850592, 0.0283...",0.121216
3126,"[thriller, drama, horror, fantasy, suicide, mu...",White Noise 2: The Light,"[0.03787311, 0.004793258, 0.014565429, 0.04266...",0.120960
4683,"[drama, comedy, horror, mumblecore, mumblegore...",Baghead,"[0.03221518, 0.0010579859, -0.010807127, 0.033...",0.120050


In [29]:
give_top_10_similar_movies(da,"The Dark Knight Rises")

,doc,original_title,doc_vec,similar
3359,"[drama, action, thriller, crime, fearless, cop...",In Too Deep,"[0.009628982, 0.016320797, 0.012850592, 0.0283...",0.121224
3073,"[action, crime, drama, thriller, police, opera...",Romeo Is Bleeding,"[0.017741414, 0.011718277, 0.0054235696, 0.016...",0.117703
3819,"[drama, action, comedy, crime, crime, fighter,...",Defendor,"[0.014526424, 0.017420372, 0.013281145, 0.0247...",0.117416
1664,"[thriller, action, crime, drama, revenge, new,...",Dead Man Down,"[-0.0055817054, 0.020834174, 0.013408287, -0.0...",0.117376
2921,"[action, comedy, crime, former, race, car, dri...",Drive Hard,"[0.030612295, 0.009799414, 0.007765317, 0.0046...",0.117206
3,"[action, crime, drama, thriller, dc, comic, cr...",The Dark Knight Rises,"[0.014835731, 0.021145223, 0.0018086568, 0.009...",0.116694
1524,"[adventure, swordplay, hero, sword, dragon, kn...",George and the Dragon,"[0.069400035, 0.03372821, -0.010562446, 0.0112...",0.116650
2388,"[action, crime, drama, thriller, revenge, murd...",I Am Wrath,"[0.021617454, 0.0110752, 0.02783181, 0.0022300...",0.115047
2212,"[action, thriller, heist, betrayal, dirty, cop...",Triple 9,"[0.024063049, 0.01940731, 0.021944573, 0.01485...",0.114233
2208,"[drama, horror, mystery, vampire, child, vampi...",Let Me In,"[0.05002707, 0.024396764, 0.0138470875, 0.0311...",0.111537
